<a href="https://colab.research.google.com/github/hdakhli/mlops-2024/blob/main/30_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#
## Word Embeddings Avec Word2Verc
https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.html

In [ ]:
!pip install nltk
!pip install gensim

# Download EBOOK ALICE'S ADVENTURES IN WONDERLAND
# https://www.gutenberg.org/files/11/11-0.txt

In [ ]:
# Lire le fichier 'alice.txt'

In [ ]:
# Remplacer les retours à la ligne \n par des espaces

In [ ]:
# Extraire chaque phrase et les mots qui la compose en un tableau de tableau à l'aide des fonctions sent_tokenize et word_tokenize de nltk.tokenize

In [ ]:
# Créer 2 modèles: CBOW et Skip Gram à partir de ce tableau. cf doc: gensim.models.Word2Vec

In [ ]:
# Comparer les mots suivants avec les 2 modelès : 
# 'alice' vs 'wonderland'
# 'alice' vs 'machines'

#
#
## LLM




In [ ]:
# Download: https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/blob/main/llama-2-7b-chat.Q2_K.gguf
!curl -O https: // huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/blob/main/llama-2-7b-chat.Q2_K.gguf

In [ ]:
!pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama

# Put the location of to the GGUF model that you've download from HuggingFace here
model_path = "llama-2-7b-chat.Q2_K.gguf"

# Create a model
model = Llama(model_path=model_path)

In [ ]:
# Prompt creation
system_message = "Vous êtes un assistant utile. Vous êtes là pour répondre uniquement à ma question, répondez uniquement en Français, répondez uniquement à la dernière question"
user_message = "Quelle est la capitale de la France ?"

prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{user_message} [/INST]"""

# Model parameters
max_tokens = 50

# Run the model
output = model(prompt, max_tokens=max_tokens, echo=True)

# Print the model output
response = output['choices'][0]['text'].split("[/INST]")[1]
print(response)

In [ ]:
user_message = response + ", Et celle de l'Espagne ? "
prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{user_message} [/INST]"""

output = model(prompt, max_tokens=max_tokens, echo=True)
response = output['choices'][0]['text'].split("[/INST]")[1]
print(response)

In [ ]:
# Que remarquez-vous ?
# Comment pourriez-vous résoudre le problème ?
# Exposer ce model en API REST avec FastAPI et Uvicorn

### LangChain

In [ ]:
!pip install langchain-core langchain-community

In [ ]:
# https://python.langchain.com/v0.1/docs/integrations/llms/llamacpp/#installation-with-windows

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q2_K.gguf",
    # temperature=0.75,
    # max_tokens=2000,
    # top_p=1,
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer only the last question in French.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
output_parser = StrOutputParser()


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
        RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
        | prompt
        | llm
        | output_parser
)

In [ ]:
from langchain_core.messages import HumanMessage

messages = []

In [ ]:
from langchain_core.messages import AIMessage
messages.append(HumanMessage(content="Quelle est la capitale de la France?"))
response = chain.invoke(
    {
        "messages": messages
    }
)

print(response)

messages.append(AIMessage(content=response))

In [ ]:
messages

In [ ]:
messages.append(HumanMessage(content="Et celle de l'Espagne ?"))
response = chain.invoke(
    {
        "messages": messages
    }
)

print(response)

messages.append(AIMessage(content=response))

In [ ]:
messages

In [ ]:
# Faites en sorte que le modèle prenne en compte l'historique de la conversation
# Exposer le model en API à l'aide de fast api et langserve

#
#
#
### Une autre alternarive
#
Télécharger et installer ollama: https://ollama.com/download

En ligne de commande, télécharger le model:
```shell
ollama pull llama2
```


In [ ]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = Ollama(model="llama2")

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Vous êtes un assistant utile. Vous êtes là pour répondre à mes questions {input}.",
        )
    ]
)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [ ]:
llm.invoke("Quelle est la capitale de la France ?")

In [ ]:
# Faites en sorte que le modèle prenne en compte l'historique de la conversation
# Exposer le model en API à l'aide de fast api et langserve